# Download and understand the data in hand

A subset of High Latitude Dust data is pre-prepared and uploaded into a s3 bucket (impact-datashare). The details can be found at https://github.com/nasa-impact/data_share .

In [ ]:
!pip install -r ../chapter-3/src/requirements.txt

In [ ]:
import boto3
import fiona

import math
import numpy as np
import os
import random
import rasterio.features

import re
import requests
import shutil

from datetime import datetime
from glob import glob
from io import BytesIO
from IPython.display import Image as Display
from PIL import Image
from rasterio.warp import calculate_default_transform


## Setup Constant variables

In [ ]:
ACCOUNT_NUMBER = "350996086543"
ROLE_NAME = "notebookAccessRole"
ROLE_ARN = f"arn:aws:iam::{ACCOUNT_NUMBER}:role/{ROLE_NAME}"
SOURCE_BUCKET = "impact-datashare"
DESTINATION_BUCKET = "<your bucket name>"

# NOTE: Use image_url function above to create a valid url, if the shapefile generation was not done in Aqua, TrueColor 
DATA_FOLDER = "data"
EVENT = "hld-labeled"
IMAGE_FOLDER = "images"
SHAPEFILE_FOLDER = "shapefiles"
URL = "https://gibs.earthdata.nasa.gov/wms/epsg4326/best/wms.cgi?SERVICE=WMS&REQUEST=GetMap&layers=MODIS_Aqua_CorrectedReflectance_TrueColor&version=1.3.0&crs=EPSG:4326&transparent=false&width={}&height={}&bbox={}&format=image/tiff&time={}"
KM_PER_DEG_AT_EQ = 111.
RESOLUTION = 0.25

## Setup environment for data transfer

In [ ]:
def assumed_role_session():
    # Assume the "notebookAccessRole" role we created using AWS CDK.
    client = boto3.client('sts')
    creds = client.assume_role(
        RoleArn=ROLE_ARN,
        RoleSessionName=ROLE_NAME
    )['Credentials']
    return boto3.session.Session(
        aws_access_key_id=creds['AccessKeyId'],
        aws_secret_access_key=creds['SecretAccessKey'],
        aws_session_token=creds['SessionToken'],
        region_name='us-east-1'
    )


## Helper methods to download and visualize data

In [ ]:
def mkdir(foldername):
    if os.path.exists(foldername):
        print(f"'{foldername}' folder already exists.")
        return
    os.makedirs(foldername)
    print(f"Created folder: {foldername}")

    
def delete_folder(foldername):
    if os.path.exists(foldername):
        shutil.rmtree(foldername) 
    else:
        print(f"Folder {foldername} doesn't exist.")
    
    
# use this if you are using anything else than Aqua and TrueColor to generate the image in the image labeler
def image_url(query_date, bbox, sensor, product, width, height):
    BASE_URL = 'https://gibs.earthdata.nasa.gov/wms/epsg4326/best/wms.cgi'
    param_dict = {
        "BBOX": bbox,
        "CRS": "EPSG:4326",
        "FORMAT": "image/jpeg",
        "HEIGHT": height,
        "LAYERS": "MODIS_%s_CorrectedReflectance_%s" % (sensor, product),
        "REQUEST": "GetMap",
        "SERVICE": "WMS",
        "TIME": query_date,
        "TRANSPARENT": "false",
        "VERSION": "1.3.0",
        "WIDTH": width,
    }

    return "{}?{}".format(BASE_URL, urlencode(param_dict))


def calculate_width_height(extent, resolution):
    """
    extent: [lower_latitude, left_longitude, higher_latitude, right_longitude], EG: [51.46162974683544,-22.94768591772153,53.03698575949367,-20.952234968354432]
    resolution: represents the pixel resolution, i.e. km/pixel. Should be a value from this list: [0.03, 0.06, 0.125, 0.25, 0.5, 1, 5, 10]
    """
    lats = extent[::2]
    lons = extent[1::2]
    km_per_deg_at_lat = KM_PER_DEG_AT_EQ * np.cos(np.pi * np.mean(lats) / 180.)
    width = int((lons[1] - lons[0]) * km_per_deg_at_lat / resolution)
    height = int((lats[1] - lats[0]) * KM_PER_DEG_AT_EQ / resolution)
    return (width, height)


def modis_url(time, extent, resolution):
    """
    time: utc time in iso format EG: 2020-02-19T00:00:00Z
    extent: [lower_latitude, left_longitude, higher_latitude, right_longitude], EG: [51.46162974683544,-22.94768591772153,53.03698575949367,-20.952234968354432]
    resolution: represents the pixel resolution, i.e. km/pixel. Should be a value from this list: [0.03, 0.06, 0.125, 0.25, 0.5, 1, 5, 10]
    """
    width, height = calculate_width_height(extent, resolution)
    extent = ','.join(map(lambda x: str(x), extent))
    return (width, height, URL.format(width, height, extent, time))


def bitmap_from_shp(fiona_shape, transform, img_shape, filename):
    """ extract out the smoke pixels using the shapefile
     from the transform defined
    Args:
        fiona_shape (Collection): fiona shape collection obtained by fiona.open()
        transfrom (rasterio.transfrom.Affine): rasterio transform object
    """
    geoms = []
    y_mtx = np.zeros((img_shape))
    for shape in fiona_shape:
        geoms.append(shape["geometry"])
    bitmap_filename = filename.replace('.shp', '_bitmap.png')
    # raster the geoms onto a bitmap
    geom_map = [(geo, 255) for geo in geoms]
    y_mtx = rasterio.features.rasterize(
        geom_map,
        out_shape=(img_shape[0], img_shape[1]),
        transform=transform
    )
    img = Image.fromarray(y_mtx)
    print(f"Preparing Bitmap: {filename}")
    img.save(f"{DATA_FOLDER}/{IMAGE_FOLDER}/{bitmap_filename}")
    

def explode(coords):
    """
    Explode a GeoJSON geometry's coordinates object and yield coordinate tuples.
    As long as the input is conforming, the type of the geometry doesn't matter.
    """
    for e in coords:
        if isinstance(e, (float, int)):
            yield coords
            break
        else:
            for f in explode(e):
                yield f


def extract_bbox(fiona_shape, offset=0):
    """
    Extract bounding box from shapefile
    """
    x, y = zip(*list(explode(fiona_shape['geometry']['coordinates'])))
    return min(y) - offset, min(x) - offset, max(y) + offset, max(x) + offset


def download_image(date, bounding_box, shapefile_name):
    """
    Download images from gibs
    date: date of event
    bounding_box: [lower_latitude, left_longitude, higher_latitude, right_longitude], EG: [51.46162974683544,-22.94768591772153,53.03698575949367,-20.952234968354432]
    """
    resolution = RESOLUTION
    width, height, url = modis_url(date, bounding_box, RESOLUTION)
    print(url)
    response = requests.get(url)
    response.raise_for_status()
    file_name = shapefile_name.replace('shp', 'tiff')
    file_name = f"{DATA_FOLDER}/{IMAGE_FOLDER}/{file_name}"
    print(f"Downloading Image: {file_name}")
    with open(file_name, 'wb') as img_file:
        img_file.write(response.content)
    return width, height, file_name


In [ ]:
# Remove already existing folder for a split and create a new one with passed filenames
def create_split(split, files):
    """
    Clear and create folder with new files.
    split: choice of "train", "test", and "val"
    files: list of tiff file paths
    """
    print(f'Preparing {split} split with {len(files)} examples.')
    folder_name = f"{DATA_FOLDER}/{split}"
    if os.path.exists(folder_name):
        delete_folder(folder_name)
    mkdir(folder_name)
    for filename in files:
        internal_filename = filename.split('/')[-1]
        bitmap_filename = filename.replace('.tiff', '_bitmap.png')
        shutil.copyfile(filename, f"{folder_name}/{internal_filename}")
        shutil.copyfile(bitmap_filename, f"{folder_name}/{bitmap_filename.split('/')[-1]}")
              

In [ ]:
# prepare train, val, and test splits
def prepare_splits(source_folder, splits={'train': 0.6, 'val': 0.2, 'test': 0.2}):
    files = glob(f"{source_folder}/*.tiff")
    print(f"Total examples found: {len(files)}")
    random.shuffle(files)
    length = len(files)
    train_limit = math.ceil(length * splits['train'])
    val_limit = train_limit + math.ceil(length * splits['train'])
    create_split('train', files[0:train_limit])
    create_split('val', files[train_limit:val_limit])
    create_split('test', files[train_limit:val_limit])
    

In [ ]:
# Download shapefiles from S3 bucket and images from WorldView.
def download_dataset(boto_session):
    """
    Download and store data in folders.
    """
    s3_connection = session.resource('s3')
    bucket = s3_connection.Bucket(SOURCE_BUCKET)
    objects = list(bucket.objects.filter(Prefix=f"{EVENT}/"))
    foldername = f"{DATA_FOLDER}/{IMAGE_FOLDER}"
    mkdir(foldername)
    for iter_object in objects:
        print(iter_object.key)
        splits = iter_object.key.split('/')
        if splits[-1]:
            filename = f"{foldername}/{splits[-1]}"
            bucket.download_file(iter_object.key, filename)


In [ ]:
def prepare_datasets(boto_session):
    """
    Download and prepare images from available shapefiles
    boto_session: Boto session currently in use.
    """
    s3_connection = session.resource('s3')
    bucket = s3_connection.Bucket(SOURCE_BUCKET)
    objects = list(bucket.objects.filter(Prefix=f"hld/"))
    foldername = f"{DATA_FOLDER}/{SHAPEFILE_FOLDER}"
    mkdir(foldername)
    for iter_object in objects:
        print(iter_object.key)
        splits = iter_object.key.split('/')
        local_foldername = f"{foldername}/{splits[1]}"
        mkdir(local_foldername)
        filename = f"{local_foldername}/{splits[-1]}"
        if not(os.path.exists(filename)):
            bucket.download_file(iter_object.key, filename)
        else: 
            print(f"File already exists. {filename}")
    mkdir(f"{DATA_FOLDER}/{IMAGE_FOLDER}")
    for shapefilename in glob(f"{foldername}/*/*.shp"):
        date = shapefilename.split('_')[1]
        filename = shapefilename.split('/')[-1]
        with fiona.open(shapefilename, "r") as shapefile:
            bounds = shapefile.bounds
            bounds = [bounds[1], bounds[0], bounds[3], bounds[2]]
            width, height, image_filename = download_image(date, bounds, filename)
            try:
                with rasterio.open(image_filename) as src:
                    bitmap_from_shp(shapefile, src.transform, (width, height), filename)
            except:
                print(f"Unable to download file: {image_filename}")
                os.remove(image_filename)


# Check downloaded data

In [ ]:
session = assumed_role_session()

In [ ]:
prepare_datasets(session)

In [ ]:
download_dataset(session)

In [ ]:
prepare_splits(f"{DATA_FOLDER}/{IMAGE_FOLDER}")

In [ ]:
!cp data ../